In [1]:
import stratigraph as sg
from mayavi import mlab
import matplotlib.pyplot as plt
import numpy as np
import h5py
from tqdm import trange
import scipy.io as sio
from PIL import ImageFont
from PIL import ImageDraw 

# set up graphics:
%matplotlib qt
plt.rcParams['svg.fonttype'] = 'none'

### Load data

In [2]:
# load data from hdf5 file (this is a 'meanderpy' model)
fname = '/Users/zoltan/Dropbox/Chronostratigraphy/meanderpy_strat_model_example_3.hdf5'
f = h5py.File(fname, 'r')
model  = f['model']
topo = np.array(model['topo'])
strat = np.array(model['strat'])
facies = np.array(model['facies'])
porosity = np.array(model['porosity'])
facies_code = {}
facies_code[int(np.array(model['point bar']))] = 'point bar'
facies_code[int(np.array(model['levee']))] = 'levee'
dx = float(np.array(model['dx']))
f.close()

In [3]:
# create facies volume (the facies data that comes from meanderpy is a 1D array and we need to expand it to 3D)
facies3d = np.zeros((strat.shape[0], strat.shape[1], strat.shape[2]-1))
for i in range(len(facies)):
    facies3d[:,:,i] = facies[i] - 1 # facies codes should start at 0

### Figure 6: Block diagram of meandering model

In [4]:
mlab.figure(bgcolor = (1,1,1))
ve = 5.0
scale = 1
dx = 10.0
bottom = np.min(strat) - 2
colors = [[0.9,0.9,0],[0.5,0.25,0]] # sand and mud facies (~ point bar and levee)
gap = 50
color_mode = 'facies'
topo_min = ve*np.min(topo[:,:,-1])
topo_max = ve*np.max(topo[:,:,-1])

sg.create_exploded_view(topo, strat, facies=facies3d, facies_colors=colors, nx=1, ny=1, gap=gap, dx=dx, ve=ve, 
    color_mode=color_mode, linewidth=0.5, bottom=bottom, opacity=1.0, x0=0, y0=0, 
    scale=1, plot_sides=True, plot_water=False, plot_surf=True, 
    surf_cmap='gist_earth', topo_min=topo_min, topo_max=topo_max, line_freq=1)

([0, 974], [0, 741])

### Figure 8: Exploded view - 9 blocks, 3 in each direction

In [5]:
mlab.clf()
ve = 10
topo_min = ve*np.min(topo[:,:,-1])
topo_max = ve*np.max(topo[:,:,-1])
temp = sg.create_exploded_view(topo, strat, facies=facies3d, facies_colors=colors, nx=3, ny=3, gap=gap, dx=dx, ve=ve, 
    color_mode=color_mode, linewidth=2, bottom=bottom, opacity=1.0, x0=0, y0=0, 
    scale=1, plot_sides=True, plot_water=False, plot_surf=True, 
    surf_cmap='gist_earth', topo_min=topo_min, topo_max=topo_max, line_freq=1)

### Figure 7: Fence diagram, property mode

In [6]:
mlab.clf()
sg.create_fence_diagram(topo, strat, prop=porosity, nx=4, ny=4, dx=dx, ve=10, 
    color_mode='property', prop_cmap = 'YlOrBr_r', prop_vmin=0, prop_vmax=0.5,
    linewidth=0.5, bottom=bottom, opacity=0.5, 
    scale=1, plot_sides=True, plot_water=False, plot_surf=True,
    surf_cmap='gist_earth', topo_min=topo_min, topo_max=topo_max, line_freq=1)
mlab.view(azimuth=27,
    elevation=60,
    distance=13000,
    focalpoint=np.array([ 5602,  3120, -1278]));

100%|█████████████████████████████████████████████| 6/6 [00:17<00:00,  2.99s/it]


### Figure 13: Time-elevation plots

In [7]:
# create time array (assumes that every point bar - overbank couplet was deposited in 5 years)
dt = 5.0
time = np.linspace(0, np.round(dt*(topo.shape[2]-1)/3), int((topo.shape[2]-1)/3)+1)

In [14]:
# only consider point bar - overbank couplets so that surfaces represent constant time increments:

elevation = topo[489, 163, ::3].copy() # single erosion + point bar; Figure 13A
# elevation = topo[400, 356, ::3].copy() # double erosion + oxbow fill; Figure 13B
# elevation = topo[354, 596, ::3].copy() # lots of erosion; Figure 13C
# elevation = topo[451, 815, ::3].copy() # single erosion + oxbow fill; Figure 13D

fig = sg.plot_strat_diagram(elevation, 'm', time, 'years', 0.05, 2.0, max(time))

### 3D Wheeler diagram

In [15]:
# use every third surface in the topography array and every second surface in the stratigraphy array:
strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo[:,:,::3], 0.05)

### Compute erosional surface attributes and plot them

In [18]:
erosional_surfs_age_below, erosional_surfs_age_above, erosional_surfs_time, erosional_surfs_thickness =\
        sg.compute_erosional_surf_attributes(strat, time, topo[:,:,::3], erosion_threshold = 0.01)

100%|█████████████████████████████████████████| 741/741 [02:35<00:00,  4.77it/s]


In [19]:
import cmocean
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, InsetPosition
from matplotlib.gridspec import GridSpec

cmap = cmocean.cm.deep_r
newcmap = cmocean.tools.crop_by_percent(cmap, 30, which='min', N=None)

def plot_strat_surface_attributes(erosional_surfs_first_age, erosional_surfs_last_age, 
                                  erosional_surfs_time, erosional_surfs_thickness, ts, cmap):
    fig = plt.figure(figsize = (18, 10))
    gs = GridSpec(2, 2, left=0.05, right=0.98, wspace=0.001)
    
    ax1 = fig.add_subplot(gs[0, 0])
    temp = erosional_surfs_first_age[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax1.imshow(temp, aspect=1, interpolation='none', cmap=cmap) #vmin=0, vmax=310)
    ax1.contour(strat[:,:,ts], colors='k', levels=20, linestyles='solid', linewidths=0.3)
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title('age of deposits below')
    cbar = fig.colorbar(im, shrink=1, ax=ax1)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('age (years)', rotation=270)
    
    ax2 = fig.add_subplot(gs[0, 1])
    temp = erosional_surfs_last_age[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax2.imshow(temp, aspect=1, interpolation='none', cmap=cmap) #vmin=0, vmax=310)
    ax2.contour(strat[:,:,ts], colors='k', levels=20, linestyles='solid', linewidths=0.3)
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_title('age of deposits above')
    cbar = fig.colorbar(im, shrink=1, ax=ax2)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('age (years)', rotation=270)
    
    ax3 = fig.add_subplot(gs[1, 0])
    temp = erosional_surfs_time[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax3.imshow(temp, aspect=1, interpolation='none', cmap=cmap) #vmin=0, vmax=310)
    ax3.contour(strat[:,:,ts], colors='k', levels=20, linestyles='solid', linewidths=0.3)
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.set_title('time gap')
    cbar = fig.colorbar(im, shrink=1, ax=ax3)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('time gap (years)', rotation=270)
    
    ax4 = fig.add_subplot(gs[1, 1])
    temp = erosional_surfs_thickness[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax4.imshow(temp, aspect=1, interpolation='none', cmap=cmap) #vmin=np.nanmin(temp), vmax=np.nanmax(temp))
    ax4.contour(strat[:,:,ts], colors='k', levels=20, linestyles='solid', linewidths=0.3)
    ax4.set_xticks([])
    ax4.set_yticks([])
    ax4.set_title('thickness eroded')
    cbar = fig.colorbar(im, shrink=1, ax=ax4)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('thickness (m)', rotation=270)
    
plot_strat_surface_attributes(erosional_surfs_age_below, erosional_surfs_age_above, 
                                  erosional_surfs_time, erosional_surfs_thickness, ts=60, cmap='viridis')

In [20]:
fig, ax1 = plt.subplots()
temp = erosional_surfs_age_below[:, :, 60].copy()
temp[temp==-1] = np.nan
im = ax1.imshow(temp, aspect=1, interpolation='none', cmap=newcmap) #vmin=0, vmax=310)
ax1.contour(strat[:,:,60], colors='k', levels=20, linestyles='solid', linewidths=0.3)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_title('age of deposits below')
cbar = fig.colorbar(im, shrink=1, ax=ax1)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('age (years)', rotation=270)

Text(0, 0.5, 'age (years)')

### Figure 9: Strike section through Wheeler diagram

In [22]:
import matplotlib as mpl
from matplotlib.colors import ListedColormap
rdbu = mpl.colormaps['RdBu'].resampled(256)
newcolors = rdbu(np.linspace(0, 1, 256))
newcolors[126:131, :] = np.array([1, 1, 1, 1])
newcmp = ListedColormap(newcolors)

# strike section
plt.figure(figsize=(15,6))
plt.imshow(wheeler[:, 775, :].T, cmap=newcmp, vmin = -4, vmax = 4, extent = [0, dx*strat.shape[0], time[-1], 0],
           interpolation='none', aspect='auto')
plt.colorbar()
plt.gca().invert_yaxis()
plt.xlabel('distance (m)', fontsize = 14)
plt.ylabel('time (years)', fontsize = 14)
plt.title('chronostratigraphic diagram', fontsize = 14)
plt.tight_layout()

In [23]:
# strike section, showing only what is preserved
plt.figure(figsize=(15,6))
plt.imshow(wheeler_strat[:, 775, :].T, cmap=newcmp, vmin = -4, vmax = 4, extent = [0, dx*strat.shape[0], time[-1], 0],
           interpolation='none', aspect='auto')
plt.gca().invert_yaxis()
plt.colorbar()
plt.xlabel('distance (m)', fontsize = 14)
plt.ylabel('time (years)', fontsize = 14)
plt.title('chronostratigraphic diagram', fontsize = 14)
plt.tight_layout()

### Figure 10: Dip section through Wheeler diagram

In [24]:
# dip section
plt.figure(figsize=(15,6))
plt.imshow(wheeler[370, :, :].T, cmap=newcmp, vmin = -6, vmax = 6, extent = [0, dx*strat.shape[1], time[-1], 0],
           interpolation='none', aspect='auto')
plt.gca().invert_yaxis()
plt.colorbar()
plt.xlabel('distance (m)', fontsize = 14)
plt.ylabel('time (years)', fontsize = 14)
plt.title('chronostratigraphic diagram', fontsize = 14)
plt.tight_layout()

In [26]:
# dip section, showing only what is preserved
plt.figure(figsize=(15,6))
plt.imshow(wheeler_strat[370, :, :].T, cmap=newcmp, vmin = -6, vmax = 6, extent = [0, dx*strat.shape[1], time[-1], 0],
           interpolation='none', aspect='auto')
plt.gca().invert_yaxis()
plt.colorbar()
plt.xlabel('distance (m)', fontsize = 14)
plt.ylabel('time (years)', fontsize = 14)
plt.title('chronostratigraphic diagram', fontsize = 14)
plt.tight_layout()

### Figure 11: Time section through Wheeler diagram

In [27]:
# time section
plt.figure(figsize=(15,10))
plt.imshow(wheeler[:, :, -1], cmap=newcmp, vmin = -6, vmax = 6, extent = [0, dx*strat.shape[1], 0, dx*strat.shape[0]],
           interpolation='none')
plt.colorbar()
x, y = np.meshgrid(np.arange(0, dx*strat.shape[1], dx), np.arange(0, dx*strat.shape[0], dx))
temp = sg.sgolay2d(topo[:,:,-1], 5, 3)
plt.contour(x, y, temp[::-1,:], colors='k', linewidths=0.5, linestyles ='solid', levels=np.arange(-12,3,2))
plt.xlabel('distance (m)', fontsize = 14)
plt.ylabel('distance (m)', fontsize = 14)
plt.title('chronostratigraphic diagram', fontsize = 14)
plt.xlim(0, dx*strat.shape[1])
plt.ylim(dx*strat.shape[0], 0)
plt.axis('equal')
# plt.tight_layout()

(0.0, 9740.0, 7410.0, 0.0)

### 3D visualization with isosurface

In [29]:
mlab.clf()
source = mlab.pipeline.scalar_field(np.swapaxes(wheeler, 0, 1))
source.spacing = [1,1,2]
mlab.pipeline.iso_surface(source, contours=[-1, 1], opacity=1, colormap='RdBu', vmin = -1.5, vmax = 1.5);

### 3D visualization with plane widgets

In [30]:
mlab.clf()
source = mlab.pipeline.scalar_field(np.swapaxes(wheeler, 0, 1))
source.spacing = [1,1,3]
for axis in ['x', 'y', 'z']:
    plane = mlab.pipeline.image_plane_widget(source, plane_orientation = '{}_axes'.format(axis),
                                           slice_index=i, colormap='RdBu', vmin = -8, vmax = 8);

### Figure 12: Stratigraphic attribute maps

In [31]:
deposition_time, erosion_time, stasis_time, vacuity_time, deposition_thickness, erosion_thickness =\
        sg.compute_strat_maps(strat, wheeler, wheeler_strat, vacuity)

In [32]:
# clip colormap (so that colors are not too dark at the lower end)
import cmocean
cmap = cmocean.cm.deep_r
# newcmap = cmap
newcmap = cmocean.tools.crop_by_percent(cmap, 10, which='min', N=None)

# elevation = topo[354, 596, ::3].copy() # lots of erosion
# elevation = topo[451, 815, ::3].copy() # single erosion + oxbow fill
# elevation = topo[489, 163, ::3].copy() # single erosion + point bar

fig, axs = plt.subplots(2, 3, sharey=True, figsize=(16, 8))
im = axs[0,0].imshow(deposition_time, vmin=0, vmax=0.3, cmap=newcmap)
# axs[0,0].contour(deposition_time, levels=np.linspace(0,0.3,10), colors='k', linewidths=0.5)
axs[0,0].set_title('deposition (time)')
axs[0,0].set_xticks([])
axs[0,0].set_yticks([])
axs[0,0].plot(596, 354, 'ro')
axs[0,0].plot(815, 451, 'ro')
axs[0,0].plot(163, 489, 'ro')
fig.colorbar(im, ax=axs[0,0], shrink=0.8)
im = axs[0,1].imshow(erosion_time, vmin=0, vmax=0.3, cmap=newcmap)
# axs[0,1].contour(erosion_time, levels=np.linspace(0,0.3,10), colors='k', linewidths=0.5)
axs[0,1].set_title('erosion (time)')
axs[0,1].set_xticks([])
axs[0,1].set_yticks([])
axs[0,1].plot(596, 354, 'ro')
axs[0,1].plot(815, 451, 'ro')
axs[0,1].plot(163, 489, 'ro')
fig.colorbar(im, ax=axs[0,1], shrink=0.8)
im = axs[0,2].imshow(stasis_time, vmin=0, vmax=1, cmap=newcmap)
# axs[0,2].contour(stasis_time, levels=np.linspace(0,1,10), colors='k', linewidths=0.5)
axs[0,2].set_title('stasis (time)')
axs[0,2].set_xticks([])
axs[0,2].set_yticks([])
axs[0,2].plot(596, 354, 'ro')
axs[0,2].plot(815, 451, 'ro')
axs[0,2].plot(163, 489, 'ro')
fig.colorbar(im, ax=axs[0,2], shrink=0.8)
im = axs[1,0].imshow(vacuity_time, vmin=0, vmax=0.3, cmap=newcmap)
# axs[1,0].contour(vacuity_time, levels=np.linspace(0,0.3,10), colors='k', linewidths=0.5)
axs[1,0].set_title('vacuity (time)')
axs[1,0].set_xticks([])
axs[1,0].set_yticks([])
axs[1,0].plot(596, 354, 'ro')
axs[1,0].plot(815, 451, 'ro')
axs[1,0].plot(163, 489, 'ro')
fig.colorbar(im, ax=axs[1,0], shrink=0.8)
im = axs[1,1].imshow(deposition_thickness, vmin=0, vmax=14, cmap=newcmap)
# axs[1,1].contour(deposition_thickness, levels=np.linspace(0,14,10), colors='k', linewidths=0.5)
axs[1,1].set_title('deposition (thickness)')
axs[1,1].set_xticks([])
axs[1,1].set_yticks([])
axs[1,1].plot(596, 354, 'ro')
axs[1,1].plot(815, 451, 'ro')
axs[1,1].plot(163, 489, 'ro')
fig.colorbar(im, ax=axs[1,1], shrink=0.8)
im = axs[1,2].imshow(-erosion_thickness, vmin=0, vmax=50, cmap=newcmap)
# axs[1,2].contour(-erosion_thickness, levels=np.linspace(0,50,10), colors='k', linewidths=0.5)
axs[1,2].set_title('erosion (thickness)')
axs[1,2].set_xticks([])
axs[1,2].set_yticks([])
axs[1,2].plot(596, 354, 'ro')
axs[1,2].plot(815, 451, 'ro')
axs[1,2].plot(163, 489, 'ro')
fig.colorbar(im, ax=axs[1,2], shrink=0.8)
fig.tight_layout()

### Plot cross sections (parts of Figures 9 and 10)

In [39]:
# data needs to be reloaded as 
# running 'sg.create_wheeler_diagram' has changed the dimensions of the 'strat' array but it did not change
# the 'facies3d' array

fname = '/Users/zoltan/Dropbox/Chronostratigraphy/meanderpy_strat_model_example_3.hdf5'
f = h5py.File(fname, 'r')
model  = f['model']
topo = np.array(model['topo'])
strat = np.array(model['strat'])
facies = np.array(model['facies'])
porosity = np.array(model['porosity'])
facies_code = {}
facies_code[int(np.array(model['point bar']))] = 'point bar'
facies_code[int(np.array(model['levee']))] = 'levee'
dx = float(np.array(model['dx']))
f.close()

In [40]:
end_time = 61
loc = 775
fig, ax = plt.subplots()
sg.plot_strike_section(topo, strat, 
                    dx, loc, ve=1, ax=ax, facies = facies3d, facies_colors=colors,
                    linewidth=0.1, line_freq=1, color_mode='facies', plot_type='2D', 
                    plot_erosion=False, erosional_surfs_thickness=erosional_surfs_thickness, 
                    plot_water=False, plot_basement=True)